# Dataset preparing


In [2]:
#data downloading

# %cd datasets
# %cd RDD2022
# !wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_Japan.zip
# !wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_India.zip
# !wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_Czech.zip
# !wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_Norway.zip
# !wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_United_States.zip
# !wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_China_MotorBike.zip
# !wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_China_Drone.zip
# !unzip RDD2022_Japan.zip
# !unzip RDD2022_India.zip
# !unzip RDD2022_Czech.zip
# !unzip RDD2022_Norway.zip
# !unzip RDD2022_United_States.zip
# !unzip RDD2022_China_Drone.zip
# !unzip RDD2022_China_MotorBike.zip
# !mv RDD2022_Japan Japan
# !mv RDD2022_India India
# !mv RDD2022_Czech Czech 
# !mv RDD2022_Norway Norway
# !mv RDD2022_United_States United_States
# !mv RDD2022_China_Drone China_Drone
# !mv RDD2022_China_MotorBike China_MotorBike

# !ls -a

/home/testdingwenchao/ShiYu_SeaView_GRDDC2022/datasets
/home/testdingwenchao/ShiYu_SeaView_GRDDC2022/datasets/RDD2022
/home/testdingwenchao/ShiYu_SeaView_GRDDC2022/datasets/RDD2022
unzip:  cannot find or open RDD2022_Japan.zip, RDD2022_Japan.zip.zip or RDD2022_Japan.zip.ZIP.
unzip:  cannot find or open RDD2022_India.zip, RDD2022_India.zip.zip or RDD2022_India.zip.ZIP.
unzip:  cannot find or open RDD2022_Czech.zip, RDD2022_Czech.zip.zip or RDD2022_Czech.zip.ZIP.
unzip:  cannot find or open RDD2022_United_States.zip, RDD2022_United_States.zip.zip or RDD2022_United_States.zip.ZIP.
unzip:  cannot find or open RDD2022_China_Drone.zip, RDD2022_China_Drone.zip.zip or RDD2022_China_Drone.zip.ZIP.
unzip:  cannot find or open RDD2022_China_MotorBike.zip, RDD2022_China_MotorBike.zip.zip or RDD2022_China_MotorBike.zip.ZIP.


# data pre-processing for yolo

In [24]:

# %pip install Pillow
# %pip install tqdm
# %cd datasets/RDD2022
# !python gene_train_val.py
# %cd ../..
# !python datasets/RDD2022/xml2yolo.py --class_file datasets/RDD2022/damage_classes.txt --input_file datasets/RDD2022/train.txt

/home/testdingwenchao/ShiYu_SeaView_GRDDC2022/datasets/RDD2022
/home/testdingwenchao/ShiYu_SeaView_GRDDC2022
total files: 38384
D00 : 13548
D10 : 7709
D40 : 3674
D20 : 8412


# data pre-processing for mmdetection

In [25]:
!python datasets/RDD2022/gene_file_list.py

100%|████████████████████████████████████| 8161/8161 [00:00<00:00, 33840.05it/s]
